# Corporate Credit Rating Dataset: Exploratory Data Analysis and Data Preparation

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings


The  [Corporate Credit Rating](https://www.kaggle.com/datasets/agewerc/corporate-credit-rating/data) dataset is obtained from Kaggle .

In [ ]:
# Load the data from .csv file 
ratings_df = pd.read_csv('./raw_corporate_rating.csv')
ratings_df.head()

The dataset `ratings_df` consists of 2029 entries (rows) and 31 columns. Each entry represents a big US firm traded on NYSE or Nasdaq. The ratings span the period from 2010 to 2016.

In [ ]:
# Check the number of entries, names of the columns and data types
ratings_df.info()

The dataset has 593 unique US firms, as seen from `ratings_df.Name.value_counts()`.  

In [ ]:
ratings_df.Name.value_counts()

## Credit Ratings

The target variable is the `Rating` column, representing the credit rating assigned by agencies. Credit ratings categorize a company's ability to repay debt. Taking a closer look at the list of agencies and their different ratings using `ratings_df['Rating Agency Name'].value_counts()` and `ratings_df.groupby('Rating Agency Name')['Rating'].unique()`:


In [ ]:
# Get the list of Rating Agency Names
ratings_df['Rating Agency Name'].value_counts()

In [ ]:
# Check the different Rating Agencies and the list of Ratings they provide
ratings_df.groupby('Rating Agency Name')['Rating'].unique()

The dataset shows an imbalance in credit ratings, with varying frequencies for each rating category as it is evident from `ratings_df.Rating.value_counts()`

In [ ]:
# Check the different types of Ratings and their counts
ratings_df.Rating.value_counts()

In [ ]:
# Plot unique ratings by agency
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
sns.countplot(x='Rating', hue='Rating Agency Name', data=ratings_df)
plt.title('Unique Ratings by Agency')

# Plot overall distribution of ratings
plt.subplot(1, 2, 2)
sns.countplot(x='Rating', data=ratings_df, order=ratings_df['Rating'].value_counts().index)
plt.title('Overall Distribution of Ratings')

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

# Address Imbalance in the Data
## Simplify and merge labels
It is also important to note that we are working with ratings from different agencies. One way to address this is to simplify and merge the ratings labels according to the following table from [Investopedia: Corporate Credit Ratings](https://www.investopedia.com/terms/c/corporate-credit-rating.asp)
| Moody's     | Standard & Poor's |  Fitch            |   Grade      | Risk         |
|-------------|-------------------|-------------------|--------------|--------------|
| Aaa         | AAA               | AAA               | Investment   | Lowest Risk  |
| Aa          | AA                | AA                | Investment   | Low Risk     |
| A           | A                 | A                 | Investment   | Low Risk     |
| Baa         | BBB               | BBB               | Investment   | Medium Risk  |
| Ba, B       | BB, B             | BB, B             | Junk         | High Risk    |
| Caa/Ca      | CCC/CC/C          | CCC/CC/C          | Junk         | Highest Risk |
| C           | D                 | D                 | Junk         | In Default   |

Instead of 10 different rating categories, we have now 6 categories. 
Using a dictionary for the mapping of new ratings and old ratings and `ratings_df['Rating'].map(rating_dict)` , we have:

In [ ]:
rating_dict = {'AAA':'Lowest Risk', 
               'AA':'Low Risk',
               'A':'Low Risk',
               'BBB':'Medium Risk', 
               'BB':'High Risk',
               'B':'High Risk',
               'CCC':'Highest Risk', 
               'CC':'Highest Risk',
               'C':'Highest Risk',
               'D':'In Default'}

ratings_df.Rating = ratings_df.Rating.map(rating_dict)
ratings_df.Rating.value_counts()

In [ ]:
plt.figure(figsize=(9, 4))
sns.countplot(x='Rating', data=ratings_df, order=ratings_df['Rating'].value_counts().index)
plt.title('Distribution of New Ratings')
plt.xlabel('Rating')
plt.ylabel('Count')
plt.show()

## Filter data

The rows with the Ratings: `'Lowest Risk` and `'In Default` are dropped from the dataset given their small value counts.

In [ ]:
ratings_df = ratings_df[ratings_df['Rating']!='Lowest Risk'] # filter Lowest Risk
ratings_df = ratings_df[ratings_df['Rating']!='In Default']  # filter In Default
ratings_df.reset_index(inplace = True, drop=True) # reset index

Although improved, our dataset still remains unbalanced.  To tackle this, SMOTE Analysis can be applied, after splitting data for train and test, to generate synthetic instances for the minority classes using the `SMOTE` function from the `imblearn.over_sampling`.  

## Input Features:
The other columns in the dataset are the input features related to financial indicators and information about the company. 

The 5 features with the company information such as `Name`, `Symbol` (for trading), `Rating Agency Name`, `Date`, and `Sector` provide context and additional details for analysis but their inclusion in the model may not be necessary for the specific task of credit rating prediction.  Different sectors exhibit distinct economic characteristics and respond differently to market conditions. By incorporating the `Sector` variable, we aim to enhance the granularity of our analysis, ensuring that the machine learning model discerns sector-specific trends and challenges.

In [ ]:
# Drop the Columns we don't want in our model input data

columns_to_drop = [ 'Name', 'Symbol', 'Rating Agency Name' ,'Date']
ratings_df = ratings_df.drop(columns=columns_to_drop)
ratings_df

# Encode categorical data

The categorical variables `Rating` and`Sector` are converted into numerical labels using the `LabelEncoder` from scikit-learn's preprocessing module, assigning a distinct integer code to each unique label. 

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

# encode Rating
le.fit(ratings_df.Rating)
ratings_df.Rating = le.transform(ratings_df.Rating) 

# encode Sector
le.fit(ratings_df.Sector)
ratings_df.Sector = le.transform(ratings_df.Sector) 

# Input Features

The dataset includes 25 financial indicators that can be categorized into different groups. These financial indicators collectively provide a comprehensive view of a company's financial health and performance, contributing to the evaluation of its creditworthiness.

**(I) Liquidity Measurement Ratios:** These ratios provide insights into a company's short-term financial health and ability to meet its immediate obligations.
1. `currentRatio`: Indicates the company's ability to cover short-term liabilities with short-term assets.
2. `quickRatio`: Measures the company's ability to cover immediate liabilities without relying on inventory.
3. `cashRatio`: Reflects the proportion of cash and cash equivalents to current liabilities.
4. `daysOfSalesOutstanding`: Measures the average number of days it takes for a company to collect payment after a sale.

**(II). Profitability Indicator Ratios:** These ratios evaluate a company's ability to generate profits relative to its revenue and investments.

5. `netProfitMargin`: Represents the percentage of profit relative to total revenue.
6. `pretaxProfitMargin`: Measures profitability before taxes are considered.
7. `grossProfitMargin`: Indicates the percentage of revenue retained after deducting the cost of goods sold.
8. `operatingProfitMargin`: Reflects the company's profitability from its core operations.
9. `returnOnAssets`: Gauges how efficiently a company utilizes its assets to generate earnings.
10. `returnOnEquity`: Measures the return generated on shareholders' equity.
11. `returnOnCapitalEmployed`: Assesses the efficiency of capital utilization in generating profits.
12. `ebitPerRevenue`: Measures earnings before interest and taxes relative to revenue.

**(III) Debt Ratios:** These ratios assess the company's leverage and debt management.

13. `debtEquityRatio`: Measures the proportion of debt relative to equity.
14. `debtRatio` : Represents the percentage of a company's assets financed by debt.

**(IV) Operating Performance Ratios:** These ratios focus on operational efficiency and effectiveness.

15. `assetTurnover`: Evaluates how efficiently a company utilizes its assets to generate sales revenue.
16. `fixedAssetTurnover` : Measures the efficiency of generating sales from fixed assets.
17. `payablesTurnover`: Measures the efficiency of a company's payment of its liabilities.

**(V) Cash Flow Indicator Ratios:** These ratios delve into a company's cash flow dynamics, providing insights into its financial sustainability. 

18. `operatingCashFlowPerShare`: Reflects the cash generated by core business operations per share.
19. `freeCashFlowPerShare`: Measures the amount of cash available to shareholders after covering operational expenses and capital expenditures.
20. `cashPerShare`: Represents the amount of cash available per outstanding share.
21. `operatingCashFlowSalesRatio`: Evaluates the percentage of sales revenue converted into cash from operating activities.
22. `freeCashFlowOperatingCashFlowRatio`: Measures the efficiency of converting operating cash flow into free cash flow.
23. `effectiveTaxRate`: Reflects the company's tax efficiency.
24. `companyEquityMultiplier`: Indicates the multiplier effect on equity due to debt
25. `enterpriseValueMultiple`: Evaluates a company's overall value relative to its earnings.

# Descriptive Statistics

The `describe()` function gives statistical descriptions like `mean`, `min`, `max`, `percentiles` of the numerical financial indicators. Comparison of the mean to the median and examining the range between percentiles, there seems to be an indication of the presence of outliers. 

In [ ]:
ratings_df.describe()

In [ ]:
# Compute the correlation matrix
correlation_matrix = ratings_df.iloc[:,1:].corr()

# Plot the correlation matrix as a heatmap
plt.figure(figsize=(12, 12))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".1f")
plt.title("Correlation Matrix of Input Features")
plt.show()

Few points can be understood from the correlation matrix:

- Companies with higher Return on Equity tend to also have higher Asset Turnover and Fixed Asset Turnover, indicating that they efficiently use their assets, both overall and fixed assets, to generate profits.

- A negative correlation between Return on Equity, Asset Turnover, and Fixed Asset Turnover with Return on Assets suggests a trade-off or inverse relationship. It may indicate that companies with high Return on Equity, efficient Asset Turnover, and effective use of Fixed Assets might not be as focused on maximizing profits from their total assets. Companies might be prioritizing shareholder returns (ROE) and operational efficiency (Asset Turnover, Fixed Asset Turnover) over maximizing profits from the entire asset base.

# Normalize the data

Features exhibit different scales, as evident from the magnitude of mean and standard deviation values. To ensure equal contribution from all features for machine learning algorithms, feature scaling is performed. The Min-Max scaling technique is applied to normalize the numerical values representing financial indicators. 

> $x_{scaled} = \frac{x - x_{min}}{x_{max} - x_{min}}$

For each column, the `MinMaxScaler` function from `sklearn.preprocessing` is used to transform the values into a standardized range between 0 and 1. The values are then multiplied by 1000, to amplify the scaled values. 

Additionally, a logarithmic transformation is applied to each value using the `np.log10` function, with a small constant (0.01) added to avoid issues with zero values. This dual transformation approach aims to normalize and potentially enhance the interpretability of the financial indicators in the dataset.

In [ ]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()

for c in ratings_df.columns[2:27]:

    ratings_df[[c]] = min_max_scaler.fit_transform(ratings_df[[c]].to_numpy())*1000
    ratings_df[[c]] = ratings_df[[c]].apply(lambda x: np.log10(x+0.01))

ratings_df.describe()

# Save the processed data

In [ ]:
ratings_df.to_csv('input_corporate_rating.csv', index=False)

This work in exploring and preparing our dataset, sets the stage for the next phase: deploying different machine learning models to forecast credit ratings.